# Kalman filter to estimate a single scalar value

In this notebook we will explore a very simple Kalman filter application. In this case, we will estimate the state of a single scalar value. This could be used, for example, in estimating the concentration of a protein. Our "motion model" is that the state changes only due to noise. In other words:

$x_{t+1} = x_t + N(0,\sigma^2)$

In plain English the value of interest $x$ at time $t+1$ is what it was previously ($x_t$) plus some random normal (also called Gaussian) noise with zero mean and variance $\sigma^2$.

We use higher dimensional numpy arrays, although this is not strictly necessary. We do this because we want to use the same style of code for higher dimensional problems, which will come next.

In [ ]:
import numpy as np
import adskalman.adskalman as adskalman
import matplotlib.pyplot as plt

In [ ]:
def column(arr):
    """convert 1D array-like to a 2D vertical array

    >>> column((1,2,3))

    array([[1],
           [2],
           [3]])
    """
    arr = np.array(arr)
    assert arr.ndim == 1
    a2 = arr[:, np.newaxis]
    return a2

In [ ]:
# Create a 1-dimensional state space model, e.g. concentration:
dt = 0.01
true_initial_state = column([0.0])
# This is F in wikipedia language.
motion_model = np.array([[1.0]])

# This is Q in wikipedia language. For a constant velocity form, it must take this specific form to be correct.
motion_noise_covariance = 1000.0*np.array([[dt]])

In [ ]:
duration = 0.5
t = np.arange(0.0, duration, dt)

In [ ]:
# Create some fake data with our model.
current_state = true_initial_state
state = []
for _ in t:
    state.append(current_state[:, 0])
    noise_sample = adskalman.rand_mvn(np.zeros(1), motion_noise_covariance, 1).T
    current_state = np.dot(motion_model, current_state) + noise_sample
state = np.array(state)

In [ ]:
plt.plot(state[:, 0], '.-', label='true values')
plt.xlabel('t')
plt.legend()
_ = plt.ylabel('x');

In [ ]:
# Create observation model. We take exactly the true value here (noise is added according to the covariance).
observation_model = np.array([[1.0]])
observation_noise_covariance = np.array([[10.0]])

In [ ]:
# Create noisy observations.
observation = []
for current_state in state:
    noise_sample = adskalman.rand_mvn(np.zeros(1), observation_noise_covariance, 1).T
    current_observation = np.dot(observation_model, column(current_state)) + noise_sample
    observation.append(current_observation[:, 0])
observation = np.array(observation)

In [ ]:
plt.plot(observation[:, 0], '.-', label='observations')
plt.xlabel('t')
plt.legend()
_ = plt.ylabel('x')

In [ ]:
# Run kalman filter on the noisy observations.
y = observation
F = motion_model
H = observation_model
Q = motion_noise_covariance
R = observation_noise_covariance
initx = true_initial_state[:, 0]
initV = 1000.0*np.eye(1)

In [ ]:
kfilt = adskalman.KalmanFilter(F, H, Q, R, initx, initV)
xfilt = []
Vfilt = []
for i, y_i in enumerate(y):
    is_initial = i == 0
    xfilt_i, Vfilt_i = kfilt.step(y=y_i, isinitial=is_initial)
    xfilt.append(xfilt_i)
    Vfilt.append(Vfilt_i)
xfilt = np.array(xfilt)
Vfilt = np.array(Vfilt)

In [ ]:
fig, axs = plt.subplots(nrows=2, figsize=(10,8))

ax = axs[0]
t = np.arange(len(xfilt[:, 0]))
low = xfilt[:, 0]-np.sqrt(Vfilt[:, 0, 0])
high = xfilt[:, 0]+np.sqrt(Vfilt[:, 0, 0])
ax.fill_between(t, low, high, alpha=0.2, color='green')

ax.plot(t,state[:, 0], '.-', label='true')
ax.plot(t,observation[:, 0], '.-', label='observed')
ax.plot(t,xfilt[:, 0], '.-', color='green', label='KF estimate')

ax.set_xlabel('t')
ax.set_ylabel('x')
ax.legend()

ax = axs[1]
ax.plot(Vfilt[:, 0, 0], '.-', label='variance')
ax.set_xlabel('t')
ax.set_ylabel('$\sigma^2$')
ax.legend();

# Q1 Note that the initial variance is quite high and then decreases. Why does it decrease after a few observations?

(Your answer should be a text explanation.)

YOUR ANSWER HERE

In [ ]:
# Now run again with missing data
y[20:30, :] = np.nan
kfilt = adskalman.KalmanFilter(F, H, Q, R, initx, initV)
xfilt = []
Vfilt = []
for i, y_i in enumerate(y):
    is_initial = i == 0
    xfilt_i, Vfilt_i = kfilt.step(y=y_i, isinitial=is_initial)
    xfilt.append(xfilt_i)
    Vfilt.append(Vfilt_i)
xfilt = np.array(xfilt)
Vfilt = np.array(Vfilt)

In [ ]:
fig, axs = plt.subplots(nrows=2, figsize=(10,8))

ax = axs[0]
t = np.arange(len(xfilt[:, 0]))
low = xfilt[:, 0]-np.sqrt(Vfilt[:, 0, 0])
high = xfilt[:, 0]+np.sqrt(Vfilt[:, 0, 0])
ax.fill_between(t, low, high, alpha=0.2, color='green')

ax.plot(t,state[:, 0], '.-', label='true')
ax.plot(t,observation[:, 0], '.-', label='observed')
ax.plot(t,xfilt[:, 0], '.-', color='green', label='KF estimate')

ax.set_xlabel('t')
ax.set_ylabel('x')
ax.legend()

ax = axs[1]
ax.plot(Vfilt[:, 0, 0], '.-', label='variance')
ax.set_xlabel('t')
ax.set_ylabel('$\sigma^2$')
ax.legend();

# Q2 Note that the variance increases between t=20 and t=30. Why is it increasing here?

(Your answer should be a text explanation.)

YOUR ANSWER HERE